<a href="https://colab.research.google.com/github/Eduardocavapent/enginernote/blob/main/C%C3%A1lculo_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Dados de entrada
T = 380  # ºK
P = 20 * 10**6  # Pa
R = 8.314  # (Pa*m^3)/(mol*K)

# EOS
a = np.array([0.175, 8.519])  # Parâmetro de interação entre misturas para cada componente
b = np.array([0.0000268, 0.000189])  # Parâmetro de interação entre mistura para cada componente

# Função Rachford Rice para calcular o resíduo
def RR(V, zi, ki):
    term_i = zi * (ki - 1) / (1 + V * (ki - 1))
    return np.sum(term_i)

# Função para resolver a Eq. Cúbica (Peng-Robinson) - Encontrar os valores de Z (Fator Compressibilidade de cada fase)
def solve_EOS(A, B):
    coefficients = [1, -(1 - B), A - 3*B**2 - 2*B, -(A*B - B**2 - B**3)]
    roots = np.roots(coefficients)
    return roots

# Função para calcular a energia de Gibbs (para selecionar a raiz correta)
def gibbs_energy(Z, A, B):
    return Z - np.log(Z - B) - A / (2 * np.sqrt(2) * B) * np.log((Z + (1 + np.sqrt(2)) * B) / (Z + (1 - np.sqrt(2)) * B))


if __name__ == "__main__":
    # RR
    V_chute = np.array([0.5,])      # Chute inicial para V
    zi = np.array([0.5, 0.5])       # Dado de entrada
    x = np.array([0.2457, 0.7443])  # Fração molar fase líquida
    y = np.array([0.7682, 0.2318])  # Fração molar fase Gás

    #V chute para comparação nos resultados
    print("V Chute:", V_chute)

    # ki:
    ki = y / x  # Coef. Equilíbrio de fases

    # Função - Encontrar o valor correto de V para RR convergir
    def objetivo(V):
        return RR(V, zi, ki)

    # Encontrar V correto
    V_correto = fsolve(objetivo, 0.5, xtol=0.2)
    print("V correto:", V_correto)

    # Encontra L
    l = (1 - V_correto)
    print("L:",l)
    print( )

    # Fase Gás:
    am_g = np.sum([y[i] * y[j] * np.sqrt(a[i] * a[j]) for i in range(len(y)) for j in range(len(y))])
    bm_g = np.sum(y * b)
    A_g = am_g * P / (R**2 * T**2)
    B_g = bm_g * P / (R * T)
    print("A do gás:", A_g)
    print("B do gás:", B_g)
    print( )

    # Fase Líquida:
    am_l = np.sum([x[i] * x[j] * np.sqrt(a[i] * a[j]) for i in range(len(x)) for j in range(len(x))])
    bm_l = np.sum(x * b)
    A_l = am_l * P / (R**2 * T**2)
    B_l = bm_l * P / (R * T)
    print("A do líquido:", A_l)
    print("B do líquido:", B_l)
    print( )

    # Fatores de Compressibilidade "Z"
    Z_v = solve_EOS(A_g, B_g)
    Z_l = solve_EOS(A_l, B_l)

    # Listas para filtrar os valores reais (Apenas as partes reais das raízes)
    Z_v_reais = [z.real for z in Z_v if np.isreal(z)]
    Z_l_reais = [z.real for z in Z_l if np.isreal(z)]

    # Verifica quantas raízes reais foram encontradas (len) e faz a seleção das raízes com base nas condições

    if len(Z_v_reais) >= 3:                       # Para as fases Vapor e Líquida >=3 raízes
        Z_v = [max(Z_v_reais), min(Z_v_reais)]
        Z_l = [max(Z_l_reais), min(Z_l_reais)]
    elif len(Z_v_reais) == 1:                     # Para as fases Vapor e Líquida == 1 raíz
        Z_v = Z_v_reais
        Z_l = Z_l_reais

    print(f"Z_v: {Z_v}")
    print(f"Z_l: {Z_l}")
    print( )

    # Energia de Gibbs para selecionar a raiz correta
    gibbs_g = [gibbs_energy(Z, A_g, B_g) for Z in Z_v]
    gibbs_l = [gibbs_energy(Z, A_l, B_l) for Z in Z_l]

    Z_g_min_gibbs = Z_v[np.argmin(gibbs_g)]
    Z_l_min_gibbs = Z_l[np.argmin(gibbs_l)]

    print("Z com menor energia de Gibbs (gás):", Z_g_min_gibbs)
    print("Z com menor energia de Gibbs (líquido):", Z_l_min_gibbs)